# How to append worktree
## Introduction
There are two ways to combine worktrees:
- Use the worktree as a node (group) inside another worktree. 
- Append the worktree directly to another worktree.

This tutorial will show an example for the second case.



In [1]:
%load_ext aiida
from aiida import load_profile
load_profile()

Profile<uuid='57ccbf7d9e2b41b39edb2bfdaf725feb' name='default'>

## Relax, Bands and PDOS
Here is an example to combine `bands` worktree, `pdos` worktree with a `relax` node to form a new worktree.

### Bands worktree

In [2]:
from aiida_quantumespresso.worktrees.bands_group import bands_worktree
from aiida import load_profile
load_profile()

bands_wt = bands_worktree(run_relax=False)
bands_wt

NodeGraphWidget(value={'name': 'WorkTree', 'uuid': 'fb426c7e-fa8c-11ee-9412-02425c27bd68', 'state': 'CREATED',…

### PDOS worktree

In [3]:
from aiida_quantumespresso.worktrees.pdos_group import pdos_worktree
pdos_wt = pdos_worktree(run_scf=True)
pdos_wt

NodeGraphWidget(value={'name': 'WorkTree', 'uuid': '10aaac7a-fa8d-11ee-9412-02425c27bd68', 'state': 'CREATED',…

### Append worktree

First, create a worktree with `relax` node:

In [4]:
from aiida_worktree import WorkTree, build_node

PwRelaxChainNode = build_node({'path': 'aiida_quantumespresso.workflows.pw.relax.PwRelaxWorkChain'})

wt = WorkTree('Electronic Structure')
relax_node = wt.nodes.new(PwRelaxChainNode, name='relax')
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': '1ff27140-fa8d-11ee-9412-02425c27bd68', 'state'…

Now let's combine `bands` worktree, `pdos` worktree with a `relax` node to form a new worktree.

To append a worktree into another worktree, there are two steps:
- append the worktree
- adjust the links, add new links between nodes, and remove old links if needed.



In [5]:
# append a wroktree
wt.append(bands_wt, prefix='bands_')
wt.append(pdos_wt, prefix='pdos_')
# adjust the links
wt.links.new(relax_node.outputs['output_structure'], wt.nodes['bands_scf'].inputs['pw.structure'])
wt.links.new(relax_node.outputs['output_structure'], wt.nodes['pdos_scf'].inputs['pw.structure'])
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': '1ff27140-fa8d-11ee-9412-02425c27bd68', 'state'…

## Node group
One can compare it with the other method: using the worktree as a node inside another worktree.



In [9]:
wt = WorkTree('Electronic Structure')
relax_node = wt.nodes.new(PwRelaxChainNode, name='relax')
bands_job = wt.nodes.new(bands_worktree, name='bands_group')
pdos_job = wt.nodes.new(pdos_worktree, name='pdos_group')
wt.links.new(relax_node.outputs['output_structure'], bands_job.inputs['structure'])
wt.links.new(relax_node.outputs['output_structure'], pdos_job.inputs['structure'])
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': 'f959a82c-fa8d-11ee-9412-02425c27bd68', 'state'…

### Pros
Show the top level workflow.
### Cons
Only know the exact node graph at runtime.